In [ ]:
import holoviews as hv
hv.notebook_extension('bokeh')

In [ ]:
from lsst.daf.persistence import Butler
butler = Butler('/project/shared/data/hsc/repo/rerun/RC/w_2018_17/DM-14055/')

In [ ]:
from lsst.qa.explorer.catalog import CoaddCatalog, VisitCatalog, MultiMatchedCatalog
from lsst.qa.explorer.utils import get_visits

tract = 9615
filt = "HSC-R"
dataId = {'tract':tract, 'filter':filt}
coaddCat = CoaddCatalog(butler, dataId)
visitCats = [VisitCatalog(butler, {'tract': tract, 'filter':filt, 'visit':v}, name=v) for v in get_visits(butler, tract, filt)]
catalog = MultiMatchedCatalog(coaddCat, visitCats, match_registry='QAmatchRegistry.h5')

In [ ]:
# Modify this cell to calculate whatever functors you want

from lsst.qa.explorer.functors import MagDiff,Seeing
gauss = MagDiff('base_GaussianFlux', 'base_PsfFlux')
seeing = Seeing()

funcs = {'gauss':gauss,'seeing':seeing,}

In [ ]:
from lsst.qa.explorer.dataset import QADataset
flags = ['calib_psfUsed',
'qaBad_flag',
'merge_measurement_i',
'merge_measurement_r',
'merge_measurement_z',
'merge_measurement_y',
'merge_measurement_g',
'base_Footprint_nPix_flag',
'base_PixelFlags_flag_inexact_psfCenter']

data = QADataset(catalog, funcs, flags=flags)

In [ ]:
# Calculate dataframe; see how long it takes. Minutes...
%time data.df.head()

In [ ]:
from lsst.qa.explorer.plots import FilterStream, multi_scattersky
filter_stream = FilterStream()
multi_scattersky(data.ds, filter_stream=filter_stream, width=900, height=300)

In [ ]:
from lsst.qa.explorer.plots import FlagSetter
import parambokeh

flag_setter = FlagSetter(filter_stream=filter_stream, flags=data.flags, bad_flags=data.flags)
parambokeh.Widgets(flag_setter, callback=flag_setter.event, push=False, on_init=True)

In [ ]:
%%output max_frames=10000
%%opts Points [width=500, height=500, tools=['hover'], colorbar=True] (cmap='coolwarm', size=4)

# Change dimension to whichever you would like to explore
dimension = 'gauss'
visit_dmap = data.visit_explore(dimension, filter_stream=filter_stream).relabel(dimension)
tap = hv.streams.Tap(source=visit_dmap, rename={'x':'ra', 'y':'dec'})

visit_dmap

In [ ]:
import lsst.afw.display
lsst.afw.display.setDefaultBackend("ginga")

from lsst.qa.explorer.display import VisitDisplay
visit_display = VisitDisplay(butler, filt, tract, dims=(500,500))
visit_display.connect_tap(tap)
visit_display.embed()